In [4]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from time import perf_counter as time
from scipy.interpolate import RegularGridInterpolator
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import scipy.ndimage
import pickle

from pygel3d import hmesh, jupyter_display as jd
import plotly.graph_objs as go
from commons.utils import *
from medial_axis_processing import unfolding, inverse_apply
from medial_axis_loader.shared import *
from medial_axis_loader import from_medial_sheet
from medial_axis_processing.volume_sampling import *
from commons.display import *

In [5]:
%load_ext autoreload
%autoreload 2

## Load volumetric data

In [25]:
volume_data_path = 'data/vindelev/X19_resampled_uint8.nii'
niiVol = nib.load(volume_data_path)

In [26]:
def rescale_max_vectorized(data, scale_factor, cval=0.0):
    pad_width = [(0, (scale_factor - dim % scale_factor) % scale_factor) for dim in data.shape]
    
    padded_data = np.pad(data, pad_width, mode='constant', constant_values=cval)
    
    new_shape = (padded_data.shape[0] // scale_factor, scale_factor,
                 padded_data.shape[1] // scale_factor, scale_factor,
                 padded_data.shape[2] // scale_factor, scale_factor)
    
    reshaped_data = padded_data.reshape(new_shape)
    reshaped_data = reshaped_data.max(axis=1).max(axis=2).max(axis=3)
    
    return reshaped_data

In [27]:
affine = niiVol.affine
imgSpacing = niiVol.header['pixdim'][1:4]

# Downsample
downsample_factor = 12
affine[0, 0] = affine[0, 0] * downsample_factor
affine[1, 1] = affine[1, 1] * downsample_factor
affine[2, 2] = affine[2, 2] * downsample_factor
imgSpacing = imgSpacing * downsample_factor

vol = niiVol.get_fdata().astype('float32') #scipy.ndimage.zoom(niiVol.get_fdata().astype('float32'), 1 / downsample_factor, order=2)
vol = rescale_max_vectorized(vol, downsample_factor)
imgDim = vol.shape

intMethod = 'linear'  # Options: "linear", "nearest", "slinear", "cubic", "quintic" and "pchip"
expVal = 0.0  # Value for extrapolation (i.e. values outside volume domain)
x = np.arange(start=0, stop=imgDim[0], step=1) * imgSpacing[0] + affine[0, 3]
y = np.arange(start=0, stop=imgDim[1], step=1) * imgSpacing[1] + affine[1, 3]
z = np.arange(start=0, stop=imgDim[2], step=1) * imgSpacing[2] + affine[2, 3]

F_vol = RegularGridInterpolator((x, y, z), vol, method=intMethod, bounds_error=False, fill_value=expVal)

In [35]:
import trimesh
threshold = 130
vertices_f, faces_f, _,_ = measure.marching_cubes(vol > threshold , 0.5)
trim = trimesh.Trimesh(vertices_f, faces_f)

connected_components = list(trim.split(only_watertight=False))
connected_components.sort(key=lambda x: len(x.faces), reverse=True)
largest_component = connected_components[0]

In [36]:
m = trimesh_to_manifold(largest_component)
jd.display(m)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([    0,    24,     2, ..., 87717, 91475, 87717]),
              'j': array([61894,     1,     1, ..., 91483, 91509, 91491]),
              'k': array([61962,     2,  2054, ..., 91491, 91476, 91500]),
              'type': 'mesh3d',
              'uid': 'af0d3561-1d6e-4388-ae2f-bc0f7d93c1d1',
              'x': array([ 4. , 94. , 93.5, ..., 59. , 60. , 60. ]),
              'y': array([ 43.5, 121. , 122. , ..., 100.5,  48.5, 100.5]),
              'z': array([69. , 48.5, 49. , ..., 30. , 55. , 20. ])},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(125,0,0)', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
              'uid': '6593b5b6-71ff-41f6-9dd4-0f5280be9b3d',
              'x': array([4.0, 4.0, None, ..., 59.5, 60.0, None], dtype=object),
              'y': array([43.5, 44.0, None, ..., 100.0, 100.5, None], dtype=obj

In [37]:
hmesh.off_save("x19_zoom_mesh.off", m)

In [19]:
smoothed = hmesh.Manifold(m)
smooth(smoothed, max_iter=3)
jd.display(smoothed)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([    0,     4,     1, ..., 91919, 91921, 91921]),
              'j': array([    9,    62,    62, ..., 91966, 91966, 91967]),
              'k': array([    1,     1,     0, ..., 91921, 91967, 91924]),
              'type': 'mesh3d',
              'uid': '65797ace-e7d8-4a47-a082-727f05d5a2f1',
              'x': array([  4.49783258,   3.94167638,   4.07722959, ..., 126.19767574,
                          126.18046809, 126.15218011]),
              'y': array([43.89654762, 44.43592161, 44.35898129, ..., 91.6476096 , 91.42575478,
                          90.68793691]),
              'z': array([68.80674981, 69.20726433, 70.81388322, ..., 54.87434807, 55.85293165,
                          56.4804361 ])},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(125,0,0)', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
              'ui